In [24]:
import tensorflow as tf #to incorporate tensors (Multi dimentional arrays)
from keras.preprocessing.image import ImageDataGenerator #to add images to dataset in real-time while the model is still training
from keras.preprocessing import image #to directly use images in our code (Since our dataset is of .jpg type and not an excel file)
from tensorflow.keras.preprocessing import image #to perform operations on images

In [25]:
from google.colab import drive
drive.mount('/content/drive') #to gain access to drive contents

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
!unzip /content/drive/MyDrive/carparts1.zip

Archive:  /content/drive/MyDrive/carparts1.zip
   creating: carparts1/
  inflating: __MACOSX/._carparts1    
   creating: carparts1/car/
  inflating: __MACOSX/carparts1/._car  
  inflating: carparts1/.DS_Store     
  inflating: __MACOSX/carparts1/._.DS_Store  
   creating: carparts1/car/single_prediction/
  inflating: __MACOSX/carparts1/car/._single_prediction  
  inflating: carparts1/car/.DS_Store  
  inflating: __MACOSX/carparts1/car/._.DS_Store  
   creating: carparts1/car/testing_set/
  inflating: __MACOSX/carparts1/car/._testing_set  
   creating: carparts1/car/training_set/
  inflating: __MACOSX/carparts1/car/._training_set  
  inflating: carparts1/car/single_prediction/002.jpg  
  inflating: __MACOSX/carparts1/car/single_prediction/._002.jpg  
  inflating: carparts1/car/single_prediction/003.jpg  
  inflating: __MACOSX/carparts1/car/single_prediction/._003.jpg  
  inflating: carparts1/car/single_prediction/001.jpg  
  inflating: __MACOSX/carparts1/car/single_prediction/._001.jpg

In [27]:
#We make the layers in CNN via code which increases the accuracy and the complexity of our model and then pass our datasets via the layers to finally reach the model for prediction.
#Epochs is the number of times we pass our dataset through the layers. As number of epochs increases, accuracy and complexity both increases.
import torch #for creating deep neural networks (PyTorch)
import torch.nn as nn #to build a basic convolution neural network (CNN) to classify the images
from torchvision.transforms import transforms #useful if we have to build a more complex transformation pipeline to transform and correlate data into a model
import numpy as np #for mathematical functions
from torch.autograd import Variable #autograd stores data and operations in a directed acyclic graph for backpropagation where variable represents nodes and backpropagation starts from here, it is generally the cost function.
from torchvision.models import squeezenet1_1 #to reduce number of parameters
import torch.functional as F
from io import open
import os
from PIL import Image #Python Imaging Library - for image processing
import pathlib
import glob #to return all file paths that match a specific pattern
from torch.utils.data import DataLoader
from torch.optim import Adam #Adaptive Moment Estimation - optimisation algorithm
import torchvision #to transform images for computer vision (In our use case)

In [28]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') #cuda uses GPU so it is preferred as it analyses the images better

In [29]:
print(device)

cpu


In [30]:
pred_path='/content/carparts1/car/single_prediction'

In [31]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])])

In [32]:
train_path='/content/carparts1/car/training_set'
test_path='/content/carparts1/car/testing_set'

In [33]:
train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [34]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()]) #iterdir - iterate directory
print(classes)

['ALTERNATOR', 'BATTERY', 'CRANKSHAFT', 'DISTRIBUTOR', 'HEADLIGHTS', 'MUFFLER']


In [35]:
#This ConvNet is a relatively simple architecture with three convolutional layers
#Each layer is followed by an activation function (ReLU).
#Batch normalization is applied after the first and third convolutional layers to stabilize the learning process.
#The network also includes a max pooling layer to reduce the spatial dimensions of the feature maps
#Then we have a fully connected layer to produce the final class scores.
class ConvNet(nn.Module): #nn.Module is a built in class in PyTorch
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()
        self.pool=nn.MaxPool2d(kernel_size=2)
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes) #fully connected layer for classification
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool(output)
        output=self.conv2(output)
        output=self.relu2(output)
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        output=output.view(-1,32*75*75)
        output=self.fc(output)
        return output

In [36]:
model=ConvNet(num_classes=6).to(device)

In [38]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001) # Adjusts the model's parameters using a learning rate of 0.001 and a weight decay of 0.0001 for regularization, to prevent overfitting.
# less learning rate makes learning slow but stable
loss_function=nn.CrossEntropyLoss() # Loss function for classification

In [39]:
num_epochs=10

In [41]:
# calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))
print(train_count,test_count)

1004 30


In [42]:
best_accuracy=0.0
for epoch in range(num_epochs):
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
        optimizer.zero_grad()
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        train_accuracy+=int(torch.sum(prediction==labels.data))
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    model.eval()
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    test_accuracy=test_accuracy/test_count
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(20.2362) Train Accuracy: 0.3386454183266932 Test Accuracy: 0.23333333333333334
Epoch: 1 Train Loss: tensor(5.8805) Train Accuracy: 0.6065737051792829 Test Accuracy: 0.6
Epoch: 2 Train Loss: tensor(3.6831) Train Accuracy: 0.704183266932271 Test Accuracy: 0.6666666666666666
Epoch: 3 Train Loss: tensor(2.1601) Train Accuracy: 0.7938247011952191 Test Accuracy: 0.6666666666666666
Epoch: 4 Train Loss: tensor(1.2799) Train Accuracy: 0.8446215139442231 Test Accuracy: 0.7666666666666667
Epoch: 5 Train Loss: tensor(1.1177) Train Accuracy: 0.8934262948207171 Test Accuracy: 0.6333333333333333
Epoch: 6 Train Loss: tensor(0.9351) Train Accuracy: 0.8834661354581673 Test Accuracy: 0.6666666666666666
Epoch: 7 Train Loss: tensor(0.3345) Train Accuracy: 0.9432270916334662 Test Accuracy: 0.7
Epoch: 8 Train Loss: tensor(0.1745) Train Accuracy: 0.9641434262948207 Test Accuracy: 0.7666666666666667
Epoch: 9 Train Loss: tensor(0.1913) Train Accuracy: 0.9561752988047809 Test Accuracy

In [49]:
def prediction(img_path,transformer):
    image=Image.open(img_path)
    image_tensor=transformer(image).float() #esulting tensor is converted to a float tensor.
    image_tensor=image_tensor.unsqueeze_(0) #Since neural networks in PyTorch expect a batch dimension (even if it's a single image), unsqueeze_ is used to add an additional dimension at position 0.
    if torch.cuda.is_available():
        image_tensor.cuda() #moved to GPU to speed up the computation
    input=Variable(image_tensor)
    output=model(input)
    index=output.data.numpy().argmax() #The output tensor is converted to a NumPy array, and argmax() is used to get the index of the highest value, which corresponds to the predicted class.
    pred=classes[index]
    return pred

In [50]:
images_path=glob.glob(pred_path+'/*.jpg')

In [51]:
'''
i.rfind('/') finds the index of the last occurrence of the '/' character in the path.
i.rfind('/') + 1 gives the index of the character immediately after the last '/'.
i[i.rfind('/')+1:] extracts the substring starting from this index to the end of the string, which is the filename.
'''
pred_dict={}
for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [52]:
print(pred_dict)

{'004.jpg': 'DISTRIBUTOR', '003.jpg': 'CRANKSHAFT', '001.jpg': 'ALTERNATOR', '002.jpg': 'BATTERY', '005.jpg': 'HEADLIGHTS', '006.jpg': 'MUFFLER'}
